This notebook runs and save an experiment. 

In [ ]:
import os

import numpy as np

from sklearn.ensemble import RandomForestRegressor


In [ ]:
os.chdir('/home/adam/adam/causal_inference')

from causal_inference.model.ols import OLS
from causal_inference.model.weighting import IPW
from causal_inference.model.blocking import Blocking
from causal_inference.experiments.run import Experiment

%load_ext autoreload
%autoreload 2

In [ ]:
SEED_VALUE = 1234

OUTCOME = 'pf_ratio_2h_8h_manual_outcome'

PATH = '/home/adam/adam/data/causal_inference/'

TRAIN_DATA_PATH = '/home/adam/adam/data/causal_inference/data/processed/guerin_2_8_train.npz'
TEST_DATA_PATH = '/home/adam/adam/data/causal_inference/data/processed/guerin_2_8_test.npz'

RESULTS_PATH = f'/home/adam/adam/data/causal_inference/results/{OUTCOME}/'

N_OF_ITERATIONS = 100

In [ ]:
np.random.seed(SEED_VALUE)

In [ ]:
# Load train data
train_data = np.load(TRAIN_DATA_PATH)
y_train, t_train, X_train = train_data['yf'], train_data['t'], train_data['x']

# Load test data
test_data = np.load(TEST_DATA_PATH)
y_test, t_test, X_test = test_data['yf'], test_data['t'], test_data['x']

In [ ]:
diff = []

for sample in range(N_OF_ITERATIONS):
    y, t = y_train[:, sample], t_train[:, sample]
    diff.append(np.mean(y[t]) - np.mean(y[~t]))

#diff = np.Series(diff)
print(np.mean(diff),
      np.percentile(diff, q=2.5, interpolation='higher'),
      np.percentile(diff, q=97.5, interpolation='lower'))

In [ ]:
diff = []

for sample in range(N_OF_ITERATIONS):
    y, t = y_test[:, sample], t_test[:, sample]
    diff.append(np.mean(y[t]) - np.mean(y[~t]))

#diff = np.Series(diff)
print(np.mean(diff),
      np.percentile(diff, q=2.5, interpolation='higher'),
      np.percentile(diff, q=97.5, interpolation='lower'))

In [ ]:
batch_of_models = [OLS(),
                   IPW(),
                   Blocking()]

In [ ]:
os.chdir(PATH + '/results/')

if not (OUTCOME in os.listdir()):
    os.makedirs(OUTCOME)

os.chdir(RESULTS_PATH)
os.getcwd()

In [ ]:
for model in batch_of_models:

    experiment = Experiment(causal_model=model,
                            n_of_iterations=N_OF_ITERATIONS)
    experiment.run(y_train=y_train, t_train=t_train, X_train=X_train,
                   y_test=y_test, t_test=t_test, X_test=X_test)

    print(experiment.summary_)

    experiment.save(RESULTS_PATH)